***building with RunnableBranch***

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gemma:2b")

model

ChatOllama(model='gemma:2b')

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser

StrOutputParser()

In [4]:
from langchain_core.prompts import PromptTemplate

prompt1 = PromptTemplate(
    template="""
        write a detailed report on the topic of: {topic}
    """,
    input_variables=['topic']
)

prompt1

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        write a detailed report on the topic of: {topic}\n    ')

In [6]:
prompt2 = PromptTemplate(
    template="""
        summarise the report {text} in short way.
    """,
    input_variables=['text']
)
prompt2

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n        summarise the report {text} in short way.\n    ')

In [7]:
from langchain_core.runnables import RunnableSequence, RunnableBranch, RunnablePassthrough

report_chain = RunnableSequence(prompt1, model, parser)

report_chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        write a detailed report on the topic of: {topic}\n    ')
| ChatOllama(model='gemma:2b')
| StrOutputParser()

In [10]:
branch_chain = RunnableBranch(
    (lambda x: len(x.split(' ')) > 500, RunnableSequence( prompt2, model, parser)),
    RunnablePassthrough()
)

branch_chain

RunnableBranch(branches=[(RunnableLambda(lambda x: len(x.split(' ')) > 500), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n        summarise the report {text} in short way.\n    ')
| ChatOllama(model='gemma:2b')
| StrOutputParser())], default=RunnablePassthrough())

In [12]:
merged_chain = RunnableSequence(report_chain, branch_chain)

merged_chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        write a detailed report on the topic of: {topic}\n    ')
| ChatOllama(model='gemma:2b')
| StrOutputParser()
| RunnableBranch(branches=[(RunnableLambda(lambda x: len(x.split(' ')) > 500), PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n        summarise the report {text} in short way.\n    ')
  | ChatOllama(model='gemma:2b')
  | StrOutputParser())], default=RunnablePassthrough())

In [11]:
user_input = input("search the topic...")


search the topic... dollars vs indian rupees


In [13]:
response = merged_chain.invoke({"topic":user_input})

response

"**Summary of the Report ## Dollars vs Indian Rupees: A Detailed Report**\n\n**Introduction:**\n\nThe report examines the differences between the US dollar and the Indian rupee in terms of their economic, political, and historical contexts. It explores the factors that influence their exchange rates, economic stability, and future prospects.\n\n**Economic Context:**\n\n* **Global Market Dynamics:** The US economy enjoys a strong and stable footing, while India's economy is undergoing significant growth.\n* **Oil Prices:** Rising oil prices impact both economies, as India is a major importer.\n* **Inflation:** Both economies struggle with inflation, but their approaches to managing it differ.\n\n**Political Context:**\n\n* **Political Stability:** Political stability is crucial for investor confidence.\n* **Government Intervention:** Both the US and India have active central banks that intervene in the foreign exchange market to influence inflation and stabilize exchange rates.\n\n**His

In [14]:
merged_chain.get_graph().print_ascii()

  +-------------+    
  | PromptInput |    
  +-------------+    
          *          
          *          
          *          
+----------------+   
| PromptTemplate |   
+----------------+   
          *          
          *          
          *          
  +------------+     
  | ChatOllama |     
  +------------+     
          *          
          *          
          *          
+-----------------+  
| StrOutputParser |  
+-----------------+  
          *          
          *          
          *          
    +--------+       
    | Branch |       
    +--------+       
          *          
          *          
          *          
  +--------------+   
  | BranchOutput |   
  +--------------+   
